## Python for REU 2019

_Burt Rosenberg, 12 May 2019_


#### Numpy,  strides, shapes and views


The <code>ndarray</code> type provided by the NumPy library is an efficient and flexible array for scientific computing. It is a Python object that represents simply shaped arrays of a homogeneous, arithmetic type. Internally, there is a C-array, and the ultimate computation will be done with processor-native addressing as in C. The representation of the array is provided to Python through the object's _shape_ and _stride_ properties.

The _stride_ of an k-dimensional array is a k-tuple, where each dimension gives the number of bytes to skip to access consecutive elements along that dimension. The _shape_ of a k-dimensional array is a k-tuple giving the number of elements in that that dimension.
<p>
C fixes the stride at compile time, and no introspective access to that stride, or way to change it. Python can introspect and reassign the stride when creating alternative _views_ of the same ndarray.
<p>
C is also unconcerned about the shape of an array. You can put arbitrary numbers into the indices and something will happen, although not always what you want to happen.

<p>
A _view_ is an alternative indexing into an array. It might or might no be a copy, depending on whether the view is possible by manipulating strides alone, or applying transpose maps to the indices. For instance, the view generated by taking every other element in the array, given by the slice notation <code>[::2]</code>, does not require a copy of the array, but simply doubles the stride in the particular dimension. The transpose operator exchanges the stride and shape values, creating a new view of the array without rewriting memory.
    
<p>
The strides for an ndarray can be retrived as the strides property of the ndarray object, and is a k-tuple of integers for a k-dimensional ndarray. The shape of an ndarray is in effect the collection of strides. A simple exercise is to call <code>reshape(-1)</code>, which will flatten the ndarray shape into a one dimensional sequence of values.



#### Arrays in C

The technology behind the ndarray is to wrap native arrays, as represented in C language, inside a Python object. To better understand ndarrays, we discuss how C handles arrays.

Computer memory is a sequence of bytes. Each byte has an address, and given an address a, the following bytes in memory are at numerically ascending addresses &mdash; a+1, a+2, etc. 

C data types are either primitive to the language, or user defined, and each requires one or multiple bytes in consecutive locations in memory for the storage of values of the type. The C keyword *sizeof* returns the number of bytes required for the type. 

The primitive types in C reflect the available hardware circuitry. The ALU (Arithmetic Logical Unit) contains circuitry that perfors operations of representations of signed or unsigned integers; the FPU (Floating Point Unit) contains circuitry that performs operations on floating point representations of real numbers.

If a primitive is stored in multiple bytes, those bytes are stored in consecutive locations in memory, begining at an address that is divisible by the data item's size. For example, a 64-bit integer is stored in 8 bytes, beginning at an address divisible by 8. This is called _data alignment_ and it could be said that this data is "long integer aligned".

<p>
In C, an array of type T is a sequence of elements of type T arranged sequentially in memory. This allows that that i-th element of the sequece be located in memory according the the formula,
<pre>
&amp;a[i] = a + i * sizeof(T).   
</pre>
For instance, if a[] is an array of int, and each int is 4 bytes, i.e. sizeof(int)==4, then the i-th element in the array is found at 4 * i bytes from the beginning of the array. C does not really know what it is accessing, but simply follows the above formula. This is compared to Python, where a data element is an object, and can be inspected. A  data element is just a sequence of bytes in memory. If i is incorrect, to large, or negative, C dutifuly goes to that location in memory and uses the next sizeof(T) bytes as a data item of type T, whether that makes sense or not.
<pre>
if in code you write
   int a[6] ;
then what is in memory is:

 int  int  int  int  int  int
+----+----+----+----+----+----+  and array such as int a[6].
^
a (the array name, a, refers to the beginning memory address)
</pre>

Multidimensional arrays, such as 
<pre>
  int a[2][3];
</pre>
are understood as a sequence of sequences. Think of it as,
<pre>
   [2]([3] int)
    ^   ^   ^
    |   |   +------------------------------------------- int (4 bytes, say) 
    |   +---------------------- a sequence of three of /
    +--- a sequence of two of /
    
what it looks like in memory
 int  int  int    int  int  int
+----+----+----+ +----+----+----+  6 int's in a row 
+--------------+ +--------------+  grouped by three at a time
</pre>
This is because just <code>int[3]</code> is a sequence of 3 integers, each integer placed after the previous in memory, then <code>int[2][3]</code> is a sequence of 2 sequences of 3 integers, each sequence of 3 integers placed after the previous in memory.

<p>
As a further example, 
<pre>
write in code:
   int a[2][3][3];
means [2]([3]([3] int)),
       ^   ^   ^   ^
       |   |   |   +---------------------------------------- int
       |   |   +---------------------------- a seq. of 3 of /
       |   +---------------- a seq. of 3 of /
       +---- a seq. of 2 of /

 +--+--+--+ +-+--+--+ +--+--+--+ +--+--+--+ +--+--+--+ +--+--+--+ seq of 3 int's
 +--------+ +-------+ +--------+ +--------+ +--------+ +--------+ seq of 3 int[3]'s
 +-----------------------------+ +------------------------------+ seq of 2 int[3][3]'s.
</pre>
<p>
Note then that in order of memory position, the rightmost index moves fastest in this C layout. That is, the elements occur from lowest address to highest as 
<pre>
a[0][0][0]
a[0][0][1]
a[0][0][2]
a[0][1][0]
a[0][1][1]
   ...
a[0][2][2]
a[1][0][0]
   ...
a[1][2][2].
</pre>


In [1]:
import numpy as np

a = np.arange(12)
b = a[::2].view()
print ("a.strides= {}, a.shape= {}\na= {}\nb.strides= {}, b.shape= {}\nb= {}\n".format(a.strides, 
    a.shape, a, b.strides, b.shape, b))
a.shape = (4,3)
b = a[::-2,::2].view()
print ("a.strides= {}, a.shape= {}\na= {}\nb.strides= {}, b.shape= {}\nb= {}\n".format(a.strides, 
    a.shape, a, b.strides, b.shape, b))
b = a.T.view()
a[0][0]=-1 # change an element in array a, and in array b as well, as it is a no-copy view of a.
print ("a.strides= {}, a.shape= {}\na= {}\nb.strides= {}, b.shape= {}\nb= {}\n".format(a.strides, 
    a.shape, a, b.strides, b.shape, b))

a.strides= (8,), a.shape= (12,)
a= [ 0  1  2  3  4  5  6  7  8  9 10 11]
b.strides= (16,), b.shape= (6,)
b= [ 0  2  4  6  8 10]

a.strides= (24, 8), a.shape= (4, 3)
a= [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
b.strides= (-48, 16), b.shape= (2, 2)
b= [[ 9 11]
 [ 3  5]]

a.strides= (24, 8), a.shape= (4, 3)
a= [[-1  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
b.strides= (8, 24), b.shape= (3, 4)
b= [[-1  3  6  9]
 [ 1  4  7 10]
 [ 2  5  8 11]]



In [2]:
import numpy as np

# no copy views. a bit different than Python slices

a = np.arange(13,dtype=float)
# in Python this would be a shallow copy
# in numpy it is a non-copy view
b = a[:]
b[0] = 37.5
print(a)

[37.5  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12. ]


#### Reshaping arrays and C vs. Fortran memory

__N.B. _this is advanced material and probably not of value to you immediately. you can skip to the next section_.__

Generally reshaping an array returns a view. The reshaping is done by manipulating the index function, leaving the data values in memory alone. However, it might require a copy, and if so, the reshape method will make a copy, but assigning directly to the reshape property will give a ValueError.

Note the difference between how slices affect Python lists and ndarrays. Where as slices of a python list copy the list, the slice notation applied to an ndarray is a _no copy view_ of the ndarray. A mapping from the slice indexes directly into the original ndarray, and changes to the slice change the original array.

By default, numpy uses a stride of _row major order_, where the rightmost index has stride 1. This is the same order as C; however Fortran uses _column major order_, where the leftmost index has stride 1.  

The shape is a tuple, and note that a singleton tuple is denoted (a,), with the comma. Else this is an integer, which will be converted in the obvious way into a tuple when in the context of a tuple.

In [3]:
# example of reshaping with and without copying

a = 1 + np.arange(6).reshape([2,3])
b = a.reshape(-1)
c = a.reshape(-1,order='F')

# because we wish to read out the elements in fortran order when inserting them sequentially in c, 
# it is not possible that c is a view of a. however, b can be. we prove this:

print("a =\n{}\nb= {}, c= {}".format(a,b,c))
a[0,0]=-1
print("a =\n{}\nb= {}, c= {}".format(a,b,c))
print("\nWe hold these truths to be self-evident: {} {}\n".format(
    b[0]==a[(0,0)], c[0]==-a[(0,0)])  # note, the true index of an array is a tuple, but syntatic sugar is available
)

a =
[[1 2 3]
 [4 5 6]]
b= [1 2 3 4 5 6], c= [1 4 2 5 3 6]
a =
[[-1  2  3]
 [ 4  5  6]]
b= [-1  2  3  4  5  6], c= [1 4 2 5 3 6]

We hold these truths to be self-evident: True True



The shape property is settable, and this leads to some simplification of the API:

In [4]:
import numpy as np ;

m = np.array([1,2,3])
print ("shape of m = ", m.shape)
print ("m=", m)
m.shape = (3,1)
print ("shape of m = ", m.shape)
print ("m=", m)


shape of m =  (3,)
m= [1 2 3]
shape of m =  (3, 1)
m= [[1]
 [2]
 [3]]


### C review

In [5]:

#include<stdio.h>
#include<stdlib.h>
#include<assert.h>

/*
 * example of multidimensional arrays
 * author: bjr
 * created: 26 sep 2017
 * last update: 
 *
 */

/*
	This program demonstrates the typing syntax for C 
	multidimensional arrays. 

	The example is: a[i][j][k] == *(k+*(j+*(k+a)))

	in this case, a is of type [][j][k] of X, where X is 
	some base type. each *(a+k) is a [][k] of X, and each
	*(*(a+k)+j) is a [] of X. 

	Finally *(*(*(a+k)+j)+i) is of type X, as is a[i][j][k].

	The typing is strong, as sizeof(*a) and sizeof(**a), will
	show. The entire number of bytes of the subsection of 
	array in the remaining dimensions are counted. And "pointer
	subtraction" will properly account for how many objects
	occur between the two pointers, not how many raw bytes.

*/

int main(int argc, char * argv[]) {

	int a[4] ;
	int i, j, k ;

	   char * acp0, * acp1 ;
		acp0 = (char *) a ;
		acp1 = (char *) (a+1) ;
	
		assert( 1==((a+1)-a)) ;
		printf( "%d: the above assert will be good\n", __LINE__) ;

		assert( sizeof(int) == (acp1-acp0)) ;
		printf( "%d: the above assert will be good\n", __LINE__) ;

		// the notation a[i] is equivalent to *(a+i)

		a[3] = 47 ;
		assert( *(a+3) == 47 ) ;
		printf( "%d: the above assert will be good\n", __LINE__) ;
		*(a+3) = 74 ;
		assert( a[3] == 74 ) ;
		printf( "%d: the above assert will be good\n", __LINE__) ;

   /*******************************/

		int m[3][2] ;

		for (i=0;i<6;i++) 
			*((*m)+i) = i ; // m is a **int, a sequence of a sequence of ints, so *m is a sequence of ints.
			// therefore, (*m)+i will be interpreted as the location of i-th integer in a sequence, with base 
			// a m. the final star *((*m)+i) reduces the type from sequene of int to int.

		for (i=0;i<3;i++)
			for (j=0;j<2;j++) printf("%d ", m[i][j]) ;
		printf("\n") ;

		int (*n)[2] ; // n is a sequence of 2 vectors of ints.
		n = m ; 

		printf("sizeof m=%lu, sizeof m[0]=%lu, sizeof m[0][0]=%lu\n", 
		sizeof(m), sizeof(m[0]), sizeof(m[0][0])) ;
		
		for (i=0;i<3;i++)
			for (j=0;j<2;j++) printf("%d ", n[i][j]) ;
		printf("\n") ;

   /*******************************/

		int r[2][3][5] ;
		printf( "sizeof(r)=%lu, sizeof(r[0])=%lu, sizeof(r[0][0])=%lu,"
			" sizeof(r[0][0][0])=%lu\n", 
		sizeof(r), sizeof(r[0]), sizeof(r[0][0]), sizeof(r[0][0][0])) ;

		// *r reduces type as does r[0], and **r as does r[0][0] 
		assert( sizeof(r[0]) == sizeof(*r) && sizeof(r[0][0]) == sizeof (**r)) ;
		printf( "%d: the above assert will be good\n", __LINE__) ;
		assert( sizeof(r[10][20][30]) == sizeof(***r)) ;
		printf( "%d: the above assert will be good\n", __LINE__) ;

	for (i=0; i< 2*3*5; i++) {
		(**r)[i] = i ;
	}

	for (i=0;i<sizeof(r)/sizeof(*r);i++)
		for(j=0;j<sizeof(*r)/sizeof(**r);j++) 
			for (k=0;k<sizeof(**r)/sizeof(***r);k++)  {
				printf("r[%d][%d][%d]=%d\n", i,j,k, r[i][j][k]) ;
				assert(r[i][j][k]==*(k+*(j+*(i+r)))) ;
			}

	return 0 ;
}

/* RUN:
    
>>$ make
cc     matrices.c   -o matrices
./matrices
43: the above assert will be good
46: the above assert will be good
52: the above assert will be good
55: the above assert will be good
0 1 2 3 4 5 
sizeof m=24, sizeof m[0]=8, sizeof m[0][0]=4
0 1 2 3 4 5 
sizeof(r)=120, sizeof(r[0])=60, sizeof(r[0][0])=20, sizeof(r[0][0][0])=4
87: the above assert will be good
89: the above assert will be good
r[0][0][0]=0
r[0][0][1]=1
r[0][0][2]=2
r[0][0][3]=3
r[0][0][4]=4
r[0][1][0]=5
r[0][1][1]=6
r[0][1][2]=7
r[0][1][3]=8
r[0][1][4]=9
r[0][2][0]=10
r[0][2][1]=11
r[0][2][2]=12
r[0][2][3]=13
r[0][2][4]=14
r[1][0][0]=15
r[1][0][1]=16
r[1][0][2]=17
r[1][0][3]=18
r[1][0][4]=19
r[1][1][0]=20
r[1][1][1]=21
r[1][1][2]=22
r[1][1][3]=23
r[1][1][4]=24
r[1][2][0]=25
r[1][2][1]=26
r[1][2][2]=27
r[1][2][3]=28
r[1][2][4]=29
>>$ 

*/

IndentationError: unexpected indent (<ipython-input-5-14e54f37bf4e>, line 7)